Data Science Capstone Assignment 2 :- Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
!pip install geocoder
import geocoder 

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = requests.get(url).text

In [3]:
soup = BeautifulSoup(req, 'xml')

In [4]:
table=soup.find('table')
col_names=['Postalcode','Borough','Neighbourhood']
df = pd.DataFrame(columns=col_names)

In [5]:
for rcell in table.find_all('tr'):
    rdata=[]
    for ccell in rcell.find_all('td'):
        rdata.append(ccell.text.strip())
    if len(rdata)==3:
        df.loc[len(df)] = rdata

In [6]:
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
df=df[df['Borough']!='Not assigned']
df.loc[df['Neighbourhood']=='Not assigned','Neighbourhood']=df['Borough']

In [8]:
df.head()

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
temp=df.groupby('Postalcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
temp=temp.reset_index(drop=False)
temp.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)

In [10]:
final = pd.merge(df, temp, on='Postalcode')
final.drop(['Neighbourhood'],axis=1,inplace=True)
final.drop_duplicates(inplace=True)
final.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)

In [11]:
final.head()

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [12]:
final.shape

(103, 3)

In [13]:
def getgeocode(postal_code):
    coords = None
    while(coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        coords = g.latlng
    lat = coords[0]
    long = coords[1]
    return lat,long

In [14]:
finaldf=pd.read_csv('http://cocl.us/Geospatial_data')

In [15]:
finaldf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
finaldf.rename(columns={'Postal Code':'Postalcode'},inplace=True)
answer = pd.merge(finaldf, final, on='Postalcode')

In [17]:
answer.head()

,Postalcode,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [18]:
answer_data=answer[['Postalcode','Borough','Neighbourhood','Latitude','Longitude']]

In [19]:
answer_data.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
